# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    #print(type(text))
    vocab_to_int = {}
    int_to_vocab = {}
    n = 0 #id应该是从0开始的吧？
    for i in range(len(text)):
        if text[i] not in vocab_to_int:
            vocab_to_int[text[i]] = n
            int_to_vocab[n] =  text[i]
            n = n+1
    return (vocab_to_int,int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {'.':'||period||',",":"||comma||",'"':'||quotation_mark||',';':'||semicolon||','!':'||exclamation_mark||',\
                 '?':'||question_mark||','(':'||left_parentheses',')':'||right_parentheses||','--':'||dash||','\n':'||return||'}
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    #知识点在3.1.9
    learning_rate = tf.placeholder(tf.float32)
    inputs = tf.placeholder(tf.int32,shape = [None,None],name = 'input')
    targets = tf.placeholder(tf.int32,shape = [None,None])

    return (inputs,targets,learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #知识点在3.1.9
    #https://github.com/tensorflow/tensorflow/issues/8191#issuecomment-292881312关于reuse的问题
    '''import inspect
    if 'reuse' in inspect.getargspec(tf.contrib.rnn.BasicLSTMCell.__init__).args:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size,reuse=True)
    else:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)'''
    
    #lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #cell = tf.contrib.rnn.MultiRNNCell([lstm]*2)
    
    cells=[]
    for _ in range(2):
        cells.append(tf.contrib.rnn.BasicLSTMCell(rnn_size))
    multicell = tf.contrib.rnn.MultiRNNCell(cells)
    initial_state = tf.identity(multicell.zero_state(batch_size, tf.float32),name='initial_state')
    return (multicell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size,embed_dim),-0.1,0.1))
    #17/09/08将random_uniform改成truncated_normal，因为似乎这个权值也是会被调整的。
    #17/09/09又改回random_uniform了，因为似乎没有什么影响
    '''
    Instead of doing the matrix multiplication, we use the weight matrix as a lookup table.
    We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094.
    Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix.
    This process is called an embedding lookup and the number of hidden units is the embedding dimension.
    The lookup table is trained just like any weight matrix as well.
    '''
    embed = tf.nn.embedding_lookup(embedding,input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    #3.10.7
    outputs,final_state = tf.nn.dynamic_rnn(cell=cell,dtype=tf.float32,inputs=inputs)
    final_state = tf.identity(final_state,'final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    inputs = get_embed(input_data,vocab_size,embed_dim)
    outputs,final_state = build_rnn(cell,inputs)
    logits = tf.contrib.layers.fully_connected(outputs,
                                               vocab_size, 
                                               activation_fn= None,
                                               weights_initializer= tf.truncated_normal_initializer(stddev=0.01))
    # 注意权重的初始化！
    # 2017/09/09 突然明白了，题目要求里的linear activation指的就是fully connected layer，因为线性函数做的事情就是 wx +b
    # 所以不需要再加入额外的激活函数。
    # 2017/09/11 大概前天理解的是错的。ReLu确实不是线性激活函数，但是activation_fn的默认值是ReLu...
    # The default value is a ReLU function. Explicitly set it to None to skip it and maintain a linear activation.
    # 但是问题来了，为什么这里要用线性激活函数呢？非线性是否减少了权重调整的空间？
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [19]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
   
    '''#3.10.6可供参考。不过并没有什么用。
    n_batches = len(int_text)//(seq_length*batch_size) 
    int_text = int_text[:n_batches*seq_length*batch_size]
    orign_batches = []
    #先做成一个list再reshape
    for n in range(n_batches):
        for nn in range(0,2):        
            for i in range(0,len(int_text),len(int_text)//batch_size):     
                for ii in range(0,seq_length):
                    #print(n,nn,i,ii)
                    if n*seq_length+nn+i+ii>=len(int_text):
                        orign_batches.append(int_text[0])
                    else:
                        orign_batches.append(int_text[i+nn+ii+n*seq_length])
    batches = np.array(orign_batches)
    #print(batches)
    batches = batches.reshape((n_batches,2,batch_size,seq_length))
    return batches
    #总结一下：有几个维度就有几个循环，一定要先分析清楚目标（batch）结构与参数的关系
    #此处有单个难点，一是batch的个数以及int_text应该drop到哪里；二是，每一个input_batch里，各组seq之间的距离；三是两个input_batch
    #之间的距离。'''
    
    # TODO: Implement Function
    # 新方法！使用np数组写出更简洁的代码！
    n_batches = int(len(int_text) / (batch_size * seq_length))

    int_text = np.array(int_text[: n_batches * batch_size * seq_length])
    target_text = np.concatenate((int_text[1:],(int_text[0],)))
    
    input_batches = np.split(int_text.reshape(batch_size, -1), n_batches, 1)
    output_batches = np.split(target_text.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(input_batches,output_batches)))
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [27]:
# Number of Epochs
num_epochs = 200
# 早期停止技术，比如如果连续20次都没有下降，就停止
# you should be able to achieve the desired loss in far few epochs. You can probably decrease your value by a factor of 10
# 从2000，减少到200
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
#[256,256]
#如果隐藏层的神经元数量太多，会看到训练集准确率远高于验证集的，那么说明你过拟合了。反之，太少的话，模型就没有足够的学习能力去处理相应的问题。
#减少神经元的数量和正则化技术可以帮助这一点，正则化技术包括弃权和L2正则化两种。好的思路是，持续添加神经元直到验证集的误差开始变大。
#一个经验是，将第一个隐藏层的神经元数设定为高于输入数，是一个好的选择
#三层隐藏层就差不多了，再往上增加得到的收益不高（卷积神经网络除外）
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.004
# 0.1~0.0000001，可以一半一半地改变，也可以10倍10倍地改变。过大会发散，过小会需要太多的echo，如果在某一块震荡，说明太大。
#一般用学习率衰减技术learning rate decay。或者自适应adaptive.
# Show stats for every n number of batches
# 17/09/11 将learning rate从0.01降低到0.005，可能可以降低 final loss
show_every_n_batches = 32

# origin: uniform: 1, fully:0.001
# (300,256,64,64,32,0.01) = 3.95
# (300,256,256,256,16,0.01) = 2.516 /然后开始每三次0.001地往下减
# (400,256,256,256,32,0.01) = 2.165/ when ehco= 300, loss= 2.18，
# (1000,256,256,256,124,0.01) = 1.83 /Epoch 400, train_loss = 2.090 结尾有挺大的振动
# (368,256,256,256,124,0.1) = 4.1 /Epoch 256, train_loss = 4.376
# 权值初始化
# (700,256,256,256,64,0.1, uniform:0.1) = 1.995 /Epoch 256, train_loss = 2.845
# (700,256,256,256,64,0.1, uniform:0.01) = 2.1 /Epoch 256, train_loss = 3.396
# (1000,256,256,256,124,0.01, uniform:0.1, fully:0.01) = 1.863  /Epoch 256,train_loss = 3.491不错，到后面还是继续在降低不像1.83
# (1800,256,256,256,100,0.1, uniform:0.1, fully:0.01) =? /Epoch 256,train_loss = 4.332太慢了，lr提高之后学习速率反而降低了= =？
# (1800,256,256,256,32,0.01, 0.1,0.01) = 振动有点大， 不如seq_length取100 的时候效果好

# 17/09/11 seq_length的长度应该与一句话的平均长度差不多（见数据探索），从100改成12.
# (200,256,256,256,12,0.004, 0.1,0.01) = 0.29
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batc h {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batc h    0/22   train_loss = 8.822
Epoch   1 Batc h   10/22   train_loss = 6.062
Epoch   2 Batc h   20/22   train_loss = 6.104
Epoch   4 Batc h    8/22   train_loss = 6.103
Epoch   5 Batc h   18/22   train_loss = 5.933
Epoch   7 Batc h    6/22   train_loss = 5.793
Epoch   8 Batc h   16/22   train_loss = 5.758
Epoch  10 Batc h    4/22   train_loss = 5.642
Epoch  11 Batc h   14/22   train_loss = 5.566
Epoch  13 Batc h    2/22   train_loss = 5.205
Epoch  14 Batc h   12/22   train_loss = 5.123
Epoch  16 Batc h    0/22   train_loss = 4.893
Epoch  17 Batc h   10/22   train_loss = 4.886
Epoch  18 Batc h   20/22   train_loss = 4.751
Epoch  20 Batc h    8/22   train_loss = 4.649
Epoch  21 Batc h   18/22   train_loss = 4.532
Epoch  23 Batc h    6/22   train_loss = 4.318
Epoch  24 Batc h   16/22   train_loss = 4.215
Epoch  26 Batc h    4/22   train_loss = 4.037
Epoch  27 Batc h   14/22   train_loss = 4.084
Epoch  29 Batc h    2/22   train_loss = 3.811
Epoch  30 Batc h   12/22   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [32]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [33]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    #print(probabilities)
    '''p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c'''
    # np.argsort的功能是返回从小到大排列的array的index，squeeze
    
    # TODO: Implement Function
    #word = int_to_vocab[np.argsort(probabilities)[:-1][0]]
    #print(int_to_vocab,probabilities)
    index = np.random.choice(range(len(probabilities)),p=probabilities)
    word = int_to_vocab[index]
    return word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [34]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: huh. i got a window here.
homer_simpson:(realizing) yeah.(half tv) oh, it is, dear great... who did you under song.
moe_szyslak:(sobs) they are been the great sleigh-horses into jerky and.
moe_szyslak: then as the moe!(quiet) quickly, the guys have a party is like an things for a dog for a romantic patterns. that's fat, wife like you might be right!
bart_simpson: nothin' who like you learn to two torn.
moe_szyslak: really?
c. :.
homer_simpson: see? x?
moe_szyslak: i can't believe it, moe. the doctor time they are a coaster and your say, i've got a guts!
moe_szyslak: i got in here.
homer_simpson:(on tape, thoughtfully) if we did some twenty little so-called anywhere on and marge... what can be it an bad voice.
moe_szyslak: i gave me a man who show him drunk?
barney_gumble: i wanna see edna without you, i don't


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.